In [3]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 100.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.0 MB/s eta 0:00:00


In [6]:
#!pip install gradio torch torchvision pillow --quiet

import gradio as gr
import torch
from torchvision import transforms, models
from PIL import Image
import numpy as np

# 1) Prepare transform & class names
def get_transform():
    return transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
    ])

class_names = [
    "Actinic Keratosis",
    "Alopecia",
    "Chondrodermatitis Nodularis",
    "Keloids",
    "Molluscum Contagiosum"
]

# 2) Load model once
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.mobilenet_v2(pretrained=False)
num_ftrs = model.classifier[1].in_features
model.classifier[1] = torch.nn.Linear(num_ftrs, len(class_names))
state = torch.load("/content/c5mobnetv2 (1).pth", map_location=device)
model.load_state_dict(state)
model.to(device).eval()

# 3) Inference function for Gradio
def predict(img: Image.Image):
    img_rgb = img.convert("RGB")
    tensor = get_transform()(img_rgb).unsqueeze(0).to(device)
    with torch.no_grad():
        logits = model(tensor)
        probs  = torch.softmax(logits, dim=1).cpu().numpy().flatten()
    # Build dict for gr.Label
    prob_dict = {class_names[i]: float(probs[i]) for i in range(len(class_names))}
    return img_rgb, prob_dict

# 4) Build and launch the interface
demo = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil", label="Upload Skin Lesion"),
    outputs=[
        gr.Image(type="pil", label="Your Input"),
        gr.Label(num_top_classes=5, label="Class Probabilities")
    ],
    title="Derma Detect",
    description="Upload a photograph of a skin disease to see the model’s prediction and probability distribution.",
    allow_flagging="never"
)

# 5) Launch immediately (share=True gives a public URL)
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e15f74ee1916c6e2ec.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
